# Module

In [54]:
import numpy as np
import xarray as xr
import pandas as pd
from netCDF4 import Dataset, MFDataset
import netCDF4
import matplotlib.pyplot as plt
import glob
from datetime import datetime, timedelta
from scipy import stats
import shutil
from scipy.spatial import distance
import metpy
import xskillscore as xs
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point
from cartopy.feature import NaturalEarthFeature
from glob import *
import sys, os, time, warnings
import pandas as pd
import pytz
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter
warnings.filterwarnings(action='ignore')
warnings.simplefilter(action='ignore')
from dask.diagnostics import ProgressBar
import metpy

# function  
아래의 함수들 포함.  

fft_1st_phase(a)  
find_nearest(array, value)  
find_nearest_idx(array, value) 
harm_1st_max_idx(array, values)  
diurnal_cycle_fig(data,daily_mean_data,time,data_source)  
grid_transfer(data)  
regrid(change_data,target_data)  
amplitude_fig(data,data_source)amplitude_fig(data,data_source)  
 print(f"latitude : {i}", end='\r') print(f"latitude : {i}", end='\r') # 순서 보여주는거


### harmonic analysis function

In [55]:
def fft_1st_phase(a):
    ff = np.fft.fft(a)
    # Get the complex vector at that frequency to retrieve amplitude and phase shift
    yy = ff[1] 

    # Calculate the amplitude
    T = a.shape[0] # domain of x; which we will divide height to get freq amplitude
    A = np.sqrt(yy.real**2 + yy.imag**2)/T 
    # print('amplitude of:', A) 
    
    # Calculate phase shift
    phi = np.arctan(yy.imag/yy.real)
    # print('phase change:', phi)
    
    return phi, yy.real ,A


def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

def find_nearest_idx(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

def harm_1st_max_idx(array, values): # 앞에가 시간 뒤에가 강수
    pp=0
    if fft_1st_phase(values)[1]>0:
        if fft_1st_phase(values)[0]<0:
            pp = -fft_1st_phase(values)[0]/np.pi *np.pi
        elif fft_1st_phase(values)[0]>0:
            pp = ( 2. - fft_1st_phase(values)[0]/np.pi ) *np.pi
    elif fft_1st_phase(values)[1]<0:
        pp = ( -fft_1st_phase(values)[0]/np.pi + 1. ) *np.pi    
    
    pp2 = np.nanmax(array,axis=0) * pp / (2.*np.pi)    
    return find_nearest_idx(array, pp2)

### diurnal cycle fig function

In [56]:
def diurnal_cycle_fig(data,daily_mean_data,time,data_source):

    if time == 'utc':
        tt = 'UTC(hour)'
    else : 
        tt = 'LST(hour)'
        
    tick_spacing = 30

    projection = ccrs.PlateCarree()
    crs = ccrs.PlateCarree()
    fig = plt.figure(figsize=(20,10))
    ax = plt.axes(projection=projection, frameon=True)


    varMin, varMax, varInt = 0, 24, 1
    levels = np.arange(varMin, varMax+varInt, varInt)
    nlevs  = levels.size
    tick_interval = 2
    cmap = plt.get_cmap('Spectral', nlevs)
    extent=[-180, 180, -60, 60]
    gl = ax.gridlines(crs=crs, draw_labels=True, linewidth=1, color='gray', alpha=1, linestyle='-.')
    gl.xlabel_style = {"size" : 13}
    gl.ylabel_style = {"size" : 13}
    gl.xlocator = mticker.FixedLocator(np.arange(extent[0], extent[1] + tick_spacing, tick_spacing))
    gl.ylocator = mticker.FixedLocator(np.arange(extent[2], extent[3] + tick_spacing, tick_spacing))

    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter()) 

    gl.top_labels = None
    gl.right_labels = None

    ax.add_feature(cfeature.COASTLINE.with_scale("50m"), lw=0.8)
    ax.add_feature(cfeature.OCEAN.with_scale("50m"), edgecolor='none', facecolor='lightgray')
    ax.add_feature(cfeature.BORDERS.with_scale("50m"), lw=0.3)
    ax.set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())

    lons = data['longitude']  # 경도
    lats = data['latitude']  # 위도
    values = data[time]

    lons, lats = np.meshgrid(data['longitude'], data['latitude'])

    cnplot = ax.contourf(lons, lats, values,cmap=cmap,levels=levels,zorder=0,transform=ccrs.PlateCarree())

    cbar = plt.colorbar(cnplot,ticks=np.arange(varMin, varMax+tick_interval, tick_interval), 
                        orientation='vertical', pad=0.01, shrink=.575) 

    zm = np.ma.masked_less(daily_mean_data, 0.275)
    plt.pcolor(lons, lats,zm, hatch='xxx', alpha=0.)

    cbar.set_label(tt, fontsize=15)
    cbar.ax.tick_params(labelsize=15)

    plt.xlabel('Longitude',fontsize=10,fontweight='bold')
    plt.ylabel('Latitude',fontsize=10,fontweight='bold')    
    ax.set_title(data_source+' [JJA 2022]',fontsize=20,fontweight='bold')
    plt.show()


### grid inverse function

In [57]:
def grid_transfer(data):
    data=data.sortby(data['latitude'], ascending=True) #위도 ㅂ반전
    data['longitude'] = xr.where(data['longitude'] > 180, data['longitude'] - 360, data['longitude']) # 경도 반전
    data = data.sortby(data['longitude']) #바뀐 경도에 대해 값들 맞춰줌 한번만 실행할것
    
    return data

### amplitude_fig

In [58]:
def amplitude_fig(data,data_source):

    tick_spacing = 30
    projection = ccrs.PlateCarree()
    crs = ccrs.PlateCarree()
    fig = plt.figure(figsize=(20,10))
    ax = plt.axes(projection=projection, frameon=True)


    varMin, varMax, varInt = 0, 2, 0.1
    levels = np.arange(varMin, varMax+varInt, varInt)
    nlevs  = levels.size
    tick_interval = 0.2
    cmap = plt.get_cmap('plasma', nlevs)
    extent=[-180, 180, -60, 60]

    gl = ax.gridlines(crs=crs, draw_labels=True, linewidth=1, color='gray', alpha=1, linestyle='-.')
    gl.xlabel_style = {"size" : 13}
    gl.ylabel_style = {"size" : 13}
    gl.xlocator = mticker.FixedLocator(np.arange(extent[0], extent[1] + tick_spacing, tick_spacing))
    gl.ylocator = mticker.FixedLocator(np.arange(extent[2], extent[3] + tick_spacing, tick_spacing))

    ax.xaxis.set_major_formatter(LongitudeFormatter())
    ax.yaxis.set_major_formatter(LatitudeFormatter()) 

    gl.top_labels = None
    gl.right_labels = None

    ax.add_feature(cfeature.COASTLINE.with_scale("50m"), lw=0.8)
    ax.add_feature(cfeature.OCEAN.with_scale("50m"), edgecolor='none', facecolor='lightgray')
    ax.add_feature(cfeature.BORDERS.with_scale("50m"), lw=0.3)
    ax.set_extent([-180, 180, -60, 60], crs=ccrs.PlateCarree())

    lons = data['longitude']  # 경도
    lats = data['latitude']  # 위도
    values = data['amlitude']

    lons, lats = np.meshgrid(data['longitude'], data['latitude'])

    cnplot = ax.contourf(lons, lats, values,cmap=cmap,levels=levels,zorder=0,
                         transform=ccrs.PlateCarree(),extend='max')

    cbar = plt.colorbar(cnplot,ticks=np.arange(varMin, varMax+tick_interval, tick_interval), 
                        orientation='vertical', pad=0.01, shrink=.56) 

    # zm = np.ma.masked_less(daily_mean_data, 0.275)
    # plt.pcolor(lons, lats,zm, hatch='xxx', alpha=0.)

    cbar.set_label('Amplitude', fontsize=15)
    cbar.ax.tick_params(labelsize=15)

    plt.xlabel('Longitude',fontsize=10,fontweight='bold')
    plt.ylabel('Latitude',fontsize=10,fontweight='bold')    
    ax.set_title(data_source+' [JJA 2022]',fontsize=20,fontweight='bold')
    plt.show()


### grid same

In [59]:
import xesmf as xe

In [60]:
def regrid(change_data,target_data):
    grid_frame = xe.Regridder(change_data, target_data, "bilinear")
    regrid_data=grid_frame(change_data)
    return regrid_data

# load lst data

In [61]:
file_list = sorted(glob('/data1/user/gychoi/Data/ERA_land/LST/'+'*.nc'))

In [62]:
file_list

['/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1979.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1980.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1981.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1982.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1983.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1984.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1985.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1986.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1987.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1988.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1989.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1990.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1991.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1992.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1993.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1994.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst1995.nc',
 '/data1/user/gychoi/Data/ERA_land/LST/ERA_lst19

In [63]:
year=list(range(1979,2023,1))

In [64]:
len(file_list)

44

In [66]:
xr.open_dataset(file_list[0])

<xarray.Dataset>
Dimensions:    (hour: 8, time: 92, latitude: 721, longitude: 1440)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1979-06-01 1979-06-02 ... 1979-08-31
  * hour       (hour) int64 0 3 6 9 12 15 18 21
Data variables:
    msshf      (hour, time, latitude, longitude) float32 ...
    mslhf      (hour, time, latitude, longitude) float32 ...
    mtpr       (hour, time, latitude, longitude) float32 ...
    ef         (hour, time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2020-11-12 09:46:32 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [16]:
for i in range (len(file_list)):
    file = xr.open_dataset(file_list[i])
    file = file.resample(time='1Y').mean('time')
    
    file.to_netcdf('/data1/user/gychoi/Data/ERA_land/yearly/era_yearly'+str(year[i])+'.nc')
    file.close()
    del(file)
    print(f'year :', year[i] ,end='\r')


In [25]:
file_list = sorted(glob('/data1/user/gychoi/Data/ERA_land/yearly/era*'))

In [26]:
file_list

['/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1979.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1980.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1981.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1982.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1983.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1984.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1985.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1986.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1987.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1988.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1989.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1990.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1991.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1992.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1993.nc',
 '/data1/user/gychoi/Data/ERA_land/yearly/era_yearly1994.nc',
 '/data1

In [31]:
for i in range (len(file_list)):
    file = xr.open_dataset(file_list[i])
    file['ef'] = file['mslhf']/(file['mslhf']+file['msshf'])
    file = file.drop_vars(['msshf','mslhf'])
    
    file.to_netcdf('/data1/user/gychoi/Data/ERA_land/real/era_yearly'+str(year[i])+'.nc')
    file.close()
    del(file)
    print(f'year :', year[i] ,end='\r')


In [32]:
file_list = sorted(glob('/data1/user/gychoi/Data/ERA_land/real/era*'))

In [35]:
file_list

['/data1/user/gychoi/Data/ERA_land/real/era_yearly1979.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1980.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1981.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1982.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1983.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1984.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1985.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1986.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1987.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1988.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1989.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1990.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1991.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1992.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1993.nc',
 '/data1/user/gychoi/Data/ERA_land/real/era_yearly1994.nc',
 '/data1/user/gychoi/Data/ERA_land/real/

In [43]:

j=0
era = xr.open_dataset(file_list[j])


longitude = era['longitude'].values
latitude = era['latitude'].values
time = era['time'].values


eralst = xr.Dataset(
    {
        'time': (['time'], time),'latitude': (['latitude'], latitude),'longitude': (['longitude'], longitude)
    }
)
eralst['efam'] = xr.DataArray(np.full((len(eralst['time']),len(eralst['latitude']), len(eralst['longitude'])), np.nan), dims=['time','latitude', 'longitude'])
eralst['pream'] = xr.DataArray(np.full((len(eralst['time']),len(eralst['latitude']), len(eralst['longitude'])), np.nan), dims=['time','latitude', 'longitude'])
eralst['prepm'] = xr.DataArray(np.full((len(eralst['time']),len(eralst['latitude']), len(eralst['longitude'])), np.nan), dims=['time','latitude', 'longitude'])

In [51]:
eralst["efam"][:,:,0]

<xarray.DataArray 'efam' (time: 1, latitude: 721)>
array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
...
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan]])
Coordinates:
  * time       (time) datetime64[ns] 1979-12-31
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    longitude  float32 0.0

In [53]:
for j in range (len(file_list)):
    era = xr.open_dataset(file_list[j])
    
    
    longitude = era['longitude'].values
    latitude = era['latitude'].values
    time = era['time'].values


    eralst = xr.Dataset(
        {
            'time': (['time'], time),'latitude': (['latitude'], latitude),'longitude': (['longitude'], longitude)
        }
    )
    eralst['efam'] = xr.DataArray(np.full((len(eralst['time']),len(eralst['latitude']), len(eralst['longitude'])), np.nan), dims=['time','latitude', 'longitude'])
    eralst['pream'] = xr.DataArray(np.full((len(eralst['time']),len(eralst['latitude']), len(eralst['longitude'])), np.nan), dims=['time','latitude', 'longitude'])
    eralst['prepm'] = xr.DataArray(np.full((len(eralst['time']),len(eralst['latitude']), len(eralst['longitude'])), np.nan), dims=['time','latitude', 'longitude'])
    
    
    
    for i in range(len(era['longitude'])):
    
        efstart=9.
        efend=(efstart+3.)

        efstart=(efstart+closest_3_hour(float(era['longitude'][i].values / 15)))
        efend=(efstart+3.)

        if(efstart<24.) and (efend>=24.):
            efstart=efstart
            efend=efend-24.

            start_index = np.where(hour == efstart)[0][0]  # 21의 인덱스를 찾습니다.
            end_index = np.where(hour == efend)[0][0]

            sliced_hour=np.concatenate((hour[start_index:],hour[:end_index+1]))

            eralst['efam'][:,:,i]=  lstmean(era['ef'],sliced_hour,i)
        elif (efstart>=24.) and (efend>=24.):
            efstart=(efstart-24.)
            efend=efend-24.

            sliced_hour=hour[np.where(hour == efstart)[0][0]:np.where(hour == efend)[0][0]+1 ]
            eralst['efam'][:,:,i]=  lstmean(era['ef'],sliced_hour,i)
        else :
            efstart=efstart
            efend=efend

            sliced_hour=hour[np.where(hour == efstart)[0][0]:np.where(hour == efend)[0][0]+1 ]
            eralst['efam'][:,:,i]=  lstmean(era['ef'],sliced_hour,i)

        print(f"efam : {i}", end='\r')
    
    
    
    for i in range(len(era['longitude'])):
    
        efstart=6.
        efend=(efstart+6.)

        efstart=(efstart+closest_3_hour(float(era['longitude'][i].values / 15)))
        efend=(efstart+3.)

        if(efstart<24.) and (efend>=24.):
            efstart=efstart
            efend=efend-24.

            start_index = np.where(hour == efstart)[0][0]  # 21의 인덱스를 찾습니다.
            end_index = np.where(hour == efend)[0][0]

            sliced_hour=np.concatenate((hour[start_index:],hour[:end_index+1]))

            eralst['pream'][:,:,i]=  lstpre(era['mtpr'],sliced_hour,i)
        elif (efstart>=24.) and (efend>=24.):
            efstart=(efstart-24.)
            efend=efend-24.

            sliced_hour=hour[np.where(hour == efstart)[0][0]:np.where(hour == efend)[0][0]+1 ]
            eralst['pream'][:,:,i]=  lstpre(era['mtpr'],sliced_hour,i)
        else :
            efstart=efstart
            efend=efend

            sliced_hour=hour[np.where(hour == efstart)[0][0]:np.where(hour == efend)[0][0]+1 ]
            eralst['pream'][:,:,i]=  lstpre(era['mtpr'],sliced_hour,i)

        print(f"pream : {i}", end='\r')
    
    
    for i in range(len(era['longitude'])):
    
        efstart=15.
        efend=(efstart+3.)

        efstart=(efstart+closest_3_hour(float(era['longitude'][i].values / 15)))
        efend=(efstart+3.)

        if(efstart<24.) and (efend>=24.):
            efstart=efstart
            efend=efend-24.

            start_index = np.where(hour == efstart)[0][0]  # 21의 인덱스를 찾습니다.
            end_index = np.where(hour == efend)[0][0]

            sliced_hour=np.concatenate((hour[start_index:],hour[:end_index+1]))

            eralst['prepm'][:,:,i]=  lstpre(era['mtpr'],sliced_hour,i)
        elif (efstart>=24.) and (efend>=24.):
            efstart=(efstart-24.)
            efend=efend-24.

            sliced_hour=hour[np.where(hour == efstart)[0][0]:np.where(hour == efend)[0][0]+1 ]
            eralst['prepm'][:,:,i]=  lstpre(era['mtpr'],sliced_hour,i)
        else :
            efstart=efstart
            efend=efend

            sliced_hour=hour[np.where(hour == efstart)[0][0]:np.where(hour == efend)[0][0]+1 ]
            eralst['prepm'][:,:,i]=  lstpre(era['mtpr'],sliced_hour,i)

        print(f"prepm : {i}", end='\r')
              
              
              
              
    eralst.to_netcdf('/data1/user/gychoi/Data/simple_tfs/yearly/eralst'+str(year[j])+'.nc')
    print(year[j],end='\r')
    eralst.close()
    era.close()
    del(eralst,era)
              
            
            
            
            
            
            
    

ef = 09시~12시  
오전 강수 = 06시 ~ 12시  
오후 강수 = 15시 ~ 21시


In [36]:
def closest_3_hour(value):
    # 가장 가까운 3시간 단위 정수 찾기
    closest = round(value / 3) * 3
    
    # 만약 24에 가깝다면 0으로 수정
    if closest == 24:
        closest = 0
        
    return closest



In [37]:
def lstmean (data, sliced_hour,isel_longitude): #variable 까지 지정해줘야함
    lst_data = data.isel(longitude = isel_longitude)
    lst_data=lst_data.sel(hour =sliced_hour ).mean('hour')
    
    return lst_data

In [38]:
hour = np.arange(0,24,3)
hour 

array([ 0,  3,  6,  9, 12, 15, 18, 21])

In [40]:
def lstpre (data, sliced_hour,isel_longitude): #variable 까지 지정해줘야함
    lst_data = data.isel(longitude = isel_longitude)
    lst_data=lst_data.sel(hour =sliced_hour ).sum('hour')
    
    return lst_data

In [50]:
with ProgressBar():
    eralst.to_netcdf('/data1/user/gychoi/Data/simple_tfs/eralst_tfs_base9397.nc')

In [51]:
print('다했다 다른거 해라')

다했다 다른거 해라
